# Use Kasumi (MISTy with sliding windows) on MIBI data

## Imports

In [ ]:
library(mistyR)
library(tidyverse)
library(future)
library(proxy)
library(withr)
library(igraph)
library(ggplot2)

In [ ]:
# Load your transformed cell table
cells = read_csv("../../data/cell_table_with_types_stage.csv")
head(cells)

In [ ]:
# Exclude bad image and SCT samples (start with "E1" or "E2")
cells <- cells %>% 
    filter(fov != "C6h") %>%
    filter(!grepl("^E[1-2]", fov))

In [ ]:
# Drop cells with "Unclear" label
# Note that we use a fixed radius in the MISTy views, so removing
# these cells should not affect the interactions between annotated cells
cells <- cells %>% filter(annotation_consensus != "Unclear")

In [ ]:
# Convert cells$Lineage to one-hot encoded columns
cells_one_hot <- cells %>%
  mutate(id = "...1") %>%  # Add an ID column to keep track of rows
  pivot_wider(names_from = annotation_consensus, values_from = annotation_consensus, values_fill = list(annotation_consensus = 0), values_fn = list(annotation_consensus = length)) %>%
  select(-id) %>% # Remove the ID column
  rename_with(~ gsub(" ", "_", .))  # Replace spaces with underscores in column names

In [ ]:
paraview_l <- 100 # Distance in pixels up to which an effect will be considered in the paraview
juxtaview_l <- 40 # Distance in pixels up to which a cell is considered a direct neighbor
panel = unique(cells$annotation_consensus)
outputfile = "misty_colorectal_lineage_multiview_no_unclear.sqm"

In [ ]:
# Expected data is a list of tibbles with each one containing the marker expressions (columns) per cell (rows) for a fov
markers <- cells_one_hot %>% group_by(fov) %>% select(all_of(panel)) %>% group_split(.keep = FALSE)

In [ ]:
# Coordinates of each cell
positions <- cells_one_hot %>% group_by(fov) %>% select(c("centroid-0", "centroid-1")) %>% group_split(.keep = FALSE)

In [ ]:
# Extract the names of the FOVs in the same order as the markers and positions tables
fov_names <- cells_one_hot %>% group_by(fov) %>% summarise(fov = first(fov)) %>% pull(fov)

In [ ]:
outputs <- seq_along(markers) %>% walk(
    function(i) {
    misty.views <- create_initial_view(markers[[i]]) %>%
        add_paraview(positions[[i]], paraview_l,
            family = "gaussian", cached = TRUE,
            prefix = "p."
        ) %>%
        add_juxtaview(positions[[i]], juxtaview_l,
            prefix = "l."
        )

    folders <- run_misty(misty.views,
        sample.id = fov_names[i],
        results.db = outputfile,
        bypass.intra = TRUE,
        cv.strict = FALSE,
        sqlite_timeout = 10000
    )})

In [ ]:
misty.results <- collect_results(outputfile)

In [ ]:
plot_view_contributions(misty.results)

In [ ]:
plot_improvement_stats(misty.results)

In [ ]:
plot_interaction_communities(misty.results, view = "juxtaview.40")

In [ ]:
plot_interaction_communities(misty.results, view = "paraview.100")

In [ ]:
plot_interaction_heatmap(misty.results, view = "paraview.100")

In [ ]:
plot_interaction_heatmap(misty.results, "paraview.100", clean = TRUE, correlation = TRUE)

In [ ]:
plot_interaction_heatmap(misty.results, view = "juxtaview.40")

In [ ]:
plot_interaction_heatmap(misty.results, "juxtaview.40", clean = TRUE, correlation = TRUE)

## Split between cancer and healthy samples

In [ ]:
library(DBI)
library(RSQLite)
library(dplyr)

In [ ]:
describe_sample_type <- function(dbfile, is_healthy = TRUE) {
  # Get all samples
  dbcon <- dbConnect(RSQLite::SQLite(), dbfile)
  samples <- dbGetQuery(dbcon, "SELECT DISTINCT sample FROM contributions") %>% 
    unlist()
  dbDisconnect(dbcon)
  
  # Create pattern based on sample type
  pattern <- if(is_healthy) {
    # Match E3 or E4 prefix
    paste0("(", paste0(
      grep("^(E3|E4)", samples, value = TRUE),
      collapse = "|"
    ), ")")
  } else {
    # Match anything not starting with E3 or E4
    paste0("(", paste0(
      grep("^(?!(E3|E4))", samples, value = TRUE, perl = TRUE),
      collapse = "|"
    ), ")")
  }
  
  # Collect filtered results
  collect_results(dbfile, pattern)
}

In [ ]:
healthy_results <- describe_sample_type(outputfile, is_healthy = TRUE)

In [ ]:
plot_view_contributions(healthy_results)
ggsave(filename = "../../figures/fig4/misty_lineage_healthy_contrib.pdf", 
        dpi = 300)

In [ ]:
tumor_results <- describe_sample_type(outputfile, is_healthy = FALSE)

In [ ]:
plot_view_contributions(tumor_results)
ggsave(filename = "../../figures/fig4/misty_lineage_tumor_contrib.pdf", 
        dpi = 300)

In [ ]:
plot_improvement_stats(healthy_results)
ggsave(filename = "../../figures/fig4/misty_lineage_healthy_improvements.pdf", dpi = 300)

In [ ]:
pdf("../../figures/fig4/misty_lineage_healthy_para_net.pdf")
plot_interaction_communities(healthy_results, view = "paraview.100")
dev.off()

In [ ]:
pdf("../../figures/fig4/misty_lineage_healthy_juxta_net.pdf")
plot_interaction_communities(healthy_results, view = "juxtaview.40")
dev.off()

In [ ]:
plot_interaction_heatmap(healthy_results, "juxtaview.40", clean = TRUE, correlation = TRUE)
plot_interaction_heatmap(healthy_results, "paraview.100", clean = TRUE, correlation = TRUE)
ggsave(filename = "../../figures/fig4/misty_lineage_healthy_para_heatmap.pdf", 
        dpi = 300)

In [ ]:
plot_improvement_stats(tumor_results)
ggsave(filename = "../../figures/fig4/misty_lineage_tumor_improvements.pdf", dpi = 300)

In [ ]:
pdf("../../figures/fig4/misty_lineage_tumor_para_net.pdf")
plot_interaction_communities(tumor_results, view = "paraview.100")
dev.off()

In [ ]:
pdf("../../figures/fig4/misty_lineage_tumor_juxta_net.pdf")
plot_interaction_communities(tumor_results, view = "juxtaview.40")
dev.off()

In [ ]:
plot_interaction_heatmap(tumor_results, "paraview.100", clean = TRUE, correlation = TRUE)
plot_interaction_heatmap(tumor_results, "juxtaview.40", clean = TRUE, correlation = TRUE)